In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn import svm

In [ ]:
df = pd.read_csv("analysis/not_uploaded/IA/KADID10k/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")
mos = pd.read_csv("analysis/not_uploaded/dmos.csv")

df["img"] = df["img"].apply(lambda row: Path(row).name)

mos = mos.rename(columns={"dist_img": "img"})

df = df.set_index("img").join(mos.set_index("img"))
df = pd.DataFrame(df.to_records())

df = df.dropna()

df["img"] = df["img"].apply(lambda row: row.split(".")[0])

df["img_id"] = df["img"].apply(lambda row: int(row.split("_")[0][1:]))
df["distortion"] = df["img"].apply(lambda row: int(row.split("_")[1]))
df["dist_level"] = df["img"].apply(lambda row: int(row.split("_")[2]))

try:
    df["score"] = (df["styles_score"] + df["technical_score"] + df["composition_score"])/3
except:
    pass
df

In [ ]:
Blurs
# 01 Gaussian blur: filter with a variable Gaussian kernel
# 02 Lens blur: filter with a circular kernel
# 03 Motion blur: ​filter with a line kernel
Color distortions
# 04 Color diffusion:  Gaussian blur the color channels (a and b) in the Lab color-space
# 05 Color shift:  randomly translate the green channel, and blend it into the original image masked by a gray level map: the normalized gradient magnitude of the original image
# 06 Color quantization: convert to indexed image using minimum variance quantization and dithering with 8 to 64 colors
# 07 Color saturation 1:  multiply the saturation channel in the HSV color-space by a factor
# 08 Color saturation 2: multiply the color channels in the Lab colorspace by a factor
Compression
# 09 JPEG2000: standard compression
# 10 JPEG: standard compression
Noise
# 11 White noise: add Gaussian white noise to the RGB image
# 12 White noise in color component: add Gaussian white noise to the YCbCr converted image (both to the luminance ‘Y‘ and the color channels ‘Cb‘ and ‘Cr‘)
# 13 Impulse noise:  add salt and pepper noise to the RGB image
# 14 Multiplicative noise: add speckle noise to the RGB image
# 15 Denoise: add Gaussian white noise to RGB image, and then apply a denoising DnCNN to each channel separately
Brightness change
# 16 Brighten: non-linearly adjust the luminance channel keeping extreme values fixed, and increasing others
# 17 Darken: similar to brighten, but decrease other values
# 18 Mean shift:  add constant to all values in image, and truncate to original value range
Spatial distortions
# 19 Jitter: randomly scatter image data by warping each pixel with random small offsets (bicubic interpolation)
# 20 Non-eccentricity patch:  randomly offset small patches in the image to nearby locations
# 21 Pixelate: downsize image and upsize it back to the original size using nearest-neighbor interpolation in each case
# 22  Quantization: quantize image values using N thresholds obtained using Otsus method
# 23  Color block: insert homogeneous random colored blocks at random locations in the image
Sharpness and contrast
# 24 High sharpen:  over-sharpen image using unsharp masking
# 25 Contrast change:  non-linearly change RGB values using a Sigmoid-type adjustment curve

In [ ]:
def histogram_distortion(distortion:int, score:str):
    plot_frame = df[(df["distortion"] == distortion) | (df["distortion"] == 0)]
    for dist_level in range(6):
        sns.distplot(plot_frame[(plot_frame["dist_level"] == dist_level)][score], label=f"{distortion}: {dist_level}")
    plt.legend()

In [ ]:
def violin_distortion(distortion:int, score:str):
    plot_frame = df[(df["distortion"] == distortion) | (df["distortion"] == 0)]
    sns.violinplot(data=plot_frame, x="dist_level", y=score, color="steelblue")
    plt.legend()

In [ ]:
histogram_distortion(10, "score")

In [ ]:
histogram_distortion(10, "dmos")

In [ ]:
violin_distortion(10, "score")

In [ ]:
violin_distortion(10, "dmos")

In [ ]:
sns.kdeplot(x=df["dmos"], y=df["score"])

In [ ]:
for dist in range(1, 26):
    results = []
    for img in range(1,82):
        results.append(stats.spearmanr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["dist_level"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["score"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))

In [ ]:
for dist in range(1, 26):
    results = []
    for img in range(1,82):
        results.append(stats.spearmanr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["dist_level"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["dmos"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))

In [ ]:
ranks = []
for dist in range(1, 26):
    results = []
    for img in range(1,82):
        results.append(stats.pearsonr(df[(df["img_id"] == img) & (df["distortion"] == dist)]["dmos"], df[(df["img_id"] == img) & (df["distortion"] == dist)]["score"]))
    print(dist,"\t", sum([val[0] for val in results])/len(results))
    ranks.append(sum([val[0] for val in results])/len(results))
print("avg", "\t", sum(ranks) / len(ranks))

In [ ]:
stats.spearmanr(df[["dmos", "score"]])

In [ ]:
stats.pearsonr(df["dmos"], df["score"])

In [ ]:
features = [c for c in df.columns if c not in ["index", "img", "img_id", "distortion", "dist_level", "var", "ref_img"]]
reg_df = df.dropna()
train = reg_df[reg_df["img_id"] <= 30].reset_index()[features]
test = reg_df[reg_df["img_id"] > 30].reset_index()[features]

In [ ]:
reg = svm.SVR().fit(train.drop(columns=["dmos"]), train["dmos"])
reg.score(train.drop(columns=["dmos"]), train["dmos"])

In [ ]:
reg.score(test.drop(columns=["dmos"]), test["dmos"])

In [ ]:
stats.pearsonr(reg.predict(test.drop(columns=["dmos"])), test["dmos"])

In [ ]:
reg = LinearRegression().fit(train.drop(columns=["dmos"]), train["dmos"])
reg.score(train.drop(columns=["dmos"]), train["dmos"])

In [ ]:
reg.score(test.drop(columns=["dmos"]), test["dmos"])

In [ ]:
reg.score(test.drop(columns=["dmos"]), test["dmos"])

In [ ]:
stats.pearsonr(reg.predict(test.drop(columns=["dmos"])), test["dmos"])

In [ ]:
sns.kdeplot(x=test["dmos"], y=reg.predict(test.drop(columns=["dmos"])))

In [ ]:
df[~df["img"].str.contains("00_0")].sort_values(by="img").reset_index()[["score","img"]].to_csv("analysis/not_uploaded/IA/TID/pred_scores", index=False, sep=" ")